In [1]:
import pandas as pd
import seaborn as sns

from Levenshtein import distance
from tqdm import tqdm

from multiprocessing import Pool
from glob import glob

# Overview 
Training data
- waltzDB


Test data
- Serrano
- AmyPro

There can't be *any* overlap (i.e. exact sequence match) between the train and test sets. Overlap between the test sets is discouraged, but not strictly disallowed

## Load data

In [2]:
path = '../../../sbxw_files/cleaned_datasets/waltzdb.csv'
waltz_df = pd.read_csv(path,index_col=0)
waltz_df = waltz_df.drop(columns='data_split')
waltz_df

,sequence,value_bool,dataset
0,STVPIE,False,waltzdb
1,GVIWIA,True,waltzdb
2,LATVYA,False,waltzdb
3,NATAHQ,False,waltzdb
4,STVGIE,False,waltzdb
...,...,...,...
1394,VNNNNN,False,waltzdb
1395,EEKKEE,False,waltzdb
1396,EKKEEK,False,waltzdb
1397,KKEEEK,False,waltzdb


In [55]:
path = '../../../sbxw_files/cleaned_datasets/serrano.csv'
serrano_df = pd.read_csv(path,index_col=0).reset_index(drop=True)
# serrano_df = serrano_df.drop(columns=['parent_protein','name','tango_pred','pH']).reset_index(drop=True)
# serrano_df.dataset = serrano_df.dataset.replace({'cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM':'serrano_s1',
#                                                  'cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM':'serrano_s2'})
# serrano_df = serrano_df.rename(columns={'exp_aggregation':'value_bool'})
# serrano_df = serrano_df.drop(columns=['data_split'])
# serrano_df['sequence_length'] = serrano_df['sequence'].apply(len)
serrano_df

,parent_protein,name,sequence,exp_aggregation,tango_pred,pH,dataset,data_split
0,t-Protein,K19,PGGGKVQIVYKPV,1,2,NaN,cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM,test
1,t-Protein,K19d,PGGGKVYKPV,0,0,NaN,cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM,test
2,t-Protein,Mut1,PGGGKNAEVYKPV,0,0,NaN,cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM,test
3,t-Protein,Mut2,PGGGKVQIVEKPV,0,0,NaN,cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM,test
4,t-Protein,K19Chym,QTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIVY,0,0,NaN,cleaned_S1_41587_2004_BFnbt1012_MOESM2_ESM,test
...,...,...,...,...,...,...,...,...
243,B-microglobulin,64-73,YLLYYTEFTP,1,2,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
244,B-microglobulin,69-78,TEFTPTEKDE,1,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
245,B-microglobulin,74-83,TEKDEYACRV,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
246,B-microglobulin,79-88,YACRVNHVTL,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test


In [4]:
path = '../../tables/250211_amypro27.csv'
amypro_df = pd.read_csv(path,index_col=0)
amypro_df = amypro_df.drop(columns=['data_split','mask']) 
amypro_df

,name,sequence,res_value_bool,dataset,value_bool,len
0,Prolactin__AP00001,LPICPGGAARCQVTLRDLFDRAVVLSHYIHNLSSEMFSEFDKRYTH...,0000001111111111111111111111111111000000001111...,amypro27,1,199
1,Alpha-s2-casein__AP00004,KNTMEHVSSSEESIISQETYKQEKNMAINPSKENLCSTFCKEVVRN...,0000000000000000000000000000000000000000000000...,amypro27,1,207
2,Serum_Amyloid_A-1__AP00005,RSFFSFLGEAFDGARDMWRAYSDMREANYIGSDKYFHARGNYDAAK...,1111111111111111111111111110000000000000000000...,amypro27,1,104
3,Medin__AP00018,RLDKQGNFNAWVAGSYGNDQWLQVDLGSSKEVTGIITQGARNFGSV...,0000000000000000000000000000000111111111111111...,amypro27,1,50
4,Apoliprotein__AP00020,TQQPQQDEMPSPTFLTQVKESLSSYWESAKTAAQNLYEKTYLPAVD...,0000000000000000000000000000000000000000000000...,amypro27,1,79
5,Odontogenic_ameloblast-associated_protein__AP...,APLIPQRLMSASNSNELLLNLNNGQLLPLQLQGPLNSWIPPFSGIL...,0000000000000000000000000000000000000000000000...,amypro27,1,264
6,Beta-lactoglobulin__AP00025,LIVTQTMKGLDIQKVAGTWYSLAMAASDISLLDAQSAPLRVYVEEL...,0000000000111111111100000000000000000000000000...,amypro27,1,162
7,Apomyoglobin__AP00030,GLSDGEWQQVLNVWGKVEADIAGHGQEVLIRLFTGHPETLEKFDKF...,1111111111111111111111111111100000000000000000...,amypro27,1,153
8,Chaplin_H__AP00039,DSGAQGAAVHSPGVLSGNVVQVPVHVPVNVCGNTISVIGLLNPAFG...,0000000000001111111111111111100111111111111111...,amypro27,1,52
9,Microcin_E492__AP00040,GETDPNTQLLNDLGNNMAWGAALGAPGGLGSAALGAAGGALQTVGQ...,0000000000000000000000000000000000000000000000...,amypro27,1,84


In [13]:
path = 'comparison_tables/amypro_vs_waltz_kmers_distance0.csv'
kmer_df = pd.read_csv(path,index_col=0)
kmer_df['seq1'] = kmer_df['index'].apply(lambda x: f'amypro{x}')
kmer_df

,index,kmer_index,kmer_seq,waltz_index,distance,seq1
750648,3,41,NFGSVQ,784,0,amypro3
1711649,8,45,GNVCIN,672,0,amypro8
1901977,10,55,GNVAVA,736,0,amypro10
2053688,11,55,IFQINS,1355,0,amypro11
3395767,17,12,GGVLVN,394,0,amypro17


In [9]:
aln_df = pd.read_csv('tables/amyproserrano_align.csv', sep=' ')
aln_df['score'] = aln_df['score'].str.replace(r'\(|\)', '', regex=True).astype(float)
aln_df

,seq0,seq1,length,score
0,serrano0,amypro0,199,19.0
1,serrano1,amypro0,199,13.0
2,serrano2,amypro0,199,15.0
3,serrano3,amypro0,199,17.0
4,serrano4,amypro0,221,14.0
...,...,...,...,...
6658,serrano243,amypro26,139,5.0
6659,serrano244,amypro26,136,10.0
6660,serrano245,amypro26,136,6.0
6661,serrano246,amypro26,137,11.0


# Find overlap

In [29]:
filt_aln_df = aln_df[(aln_df['seq1'].isin(kmer_df['seq1']))&(aln_df['score']>40)].copy(deep=True)
filt_aln_df['serrano_index'] = filt_aln_df['seq0'].apply(lambda x: int(x[7:]))
filt_aln_df['amypro_index'] = filt_aln_df['seq1'].apply(lambda x: int(x[6:]))
filt_aln_df

,seq0,seq1,length,score,serrano_index,amypro_index
2923,serrano208,amypro11,130,51.0,208,11
2924,serrano209,amypro11,130,51.0,209,11
2925,serrano210,amypro11,130,53.0,210,11
2926,serrano211,amypro11,130,56.0,211,11
2927,serrano212,amypro11,130,63.0,212,11
2928,serrano213,amypro11,130,61.0,213,11
2929,serrano214,amypro11,130,53.0,214,11
2930,serrano215,amypro11,130,54.0,215,11
2931,serrano216,amypro11,130,56.0,216,11
2932,serrano217,amypro11,130,52.0,217,11


In [30]:
len(filt_aln_df)

24

In [31]:
amypro_df.iloc[11]

name                                            Lysozyme_C__AP00016
sequence          KVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYN...
res_value_bool    0000111111111100000000001111111111000000000000...
dataset                                                    amypro27
value_bool                                                        1
len                                                             130
Name: 11, dtype: object

In [32]:
serrano_df.iloc[filt_aln_df.serrano_index]

,parent_protein,name,sequence,exp_aggregation,tango_pred,pH,dataset,data_split
31,human lysozyme,5-14,RCELARTLKR,1,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
32,human lysozyme,10-19,RTLKRLGMDG,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
33,human lysozyme,15-24,LGMDGYRGIS,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
34,human lysozyme,20-29,YRGISLANWM,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
35,human lysozyme,25-34,LANWMCLAKW,1,1,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
36,human lysozyme,30-39,CLAKWESGYN,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
37,human lysozyme,35-44,ESGYNTRATN,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
38,human lysozyme,40-49,TRATNYNAGD,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
39,human lysozyme,45-54,YNAGDRSTDY,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test
40,human lysozyme,50-59,RSTDYGIFQI,0,0,NaN,cleaned_S2_41587_2004_BFnbt1012_MOESM3_ESM,test


In [67]:
serrano_df.iloc[filt_aln_df.serrano_index].parent_protein.iloc[0]

'human lysozyme'

In [51]:
amypro_df[amypro_df.index.isin(filt_aln_df['amypro_index'].unique())]

,name,sequence,res_value_bool,dataset,value_bool,len
11,Lysozyme_C__AP00016,KVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYN...,0000111111111100000000001111111111000000000000...,amypro27,1,130


# Create filtered versions of the serrano dataset

In [57]:
path = 'tables/serrano_filteredlysozyme.csv'

print(len(serrano_df))
filt_serrano_df = serrano_df.loc[~serrano_df.index.isin(filt_aln_df.serrano_index)].reset_index()
print(len(filt_serrano_df))
filt_serrano_df.to_csv(path)

248
224
